In [13]:
import sys
import os
sys.path.append('../')
from pycore.tikzeng import *
from pycore.blocks  import *

# Generate 3D U-Net Tex Code

In [ ]:
# Define the architecture diagram list
arch = [ 
    to_head('..'), 
    to_cor(),
    to_begin(),
    
    # ---------------------------------------------------------
    # Input
    # ---------------------------------------------------------
    # Input: 5-frame sequence image
    to_input( '../inputpic/plant03_day01.png', to='(-2,0,0)', width=8, height=8, name="input" ),

    # ---------------------------------------------------------
    # Encoder (Downsampling)
    # ---------------------------------------------------------
    # Block-1: DoubleConv3D (3 -> 64)
    to_ConvConvRelu( name='ccr_b1', s_filer=256, n_filer=(64,64), offset="(0,0,0)", to="(0,0,0)", width=(2,2), height=40, depth=40, caption="Input Conv" ),
    to_Pool(name="pool_b1", offset="(0,0,0)", to="(ccr_b1-east)", width=1, height=32, depth=32, opacity=0.5),
    
    # Block-2: Down 1 (64 -> 128)
    *block_2ConvPool( name='b2', botton='pool_b1', top='pool_b2', s_filer=128, n_filer=128, offset="(1,0,0)", size=(32,32,3.5), opacity=0.5 ),
    
    # Block-3: Down 2 (128 -> 256)
    *block_2ConvPool( name='b3', botton='pool_b2', top='pool_b3', s_filer=64, n_filer=256, offset="(1,0,0)", size=(25,25,4.5), opacity=0.5 ),
    
    # Block-4: Down 3 (256 -> 512)
    *block_2ConvPool( name='b4', botton='pool_b3', top='pool_b4', s_filer=32,  n_filer=512, offset="(1,0,0)", size=(16,16,5.5), opacity=0.5 ),

    # ---------------------------------------------------------
    # Bottleneck
    # ---------------------------------------------------------
    # Block-5: Down 4 (512 -> 512)
    to_ConvConvRelu( name='ccr_b5', s_filer=16, n_filer=(512,512), offset="(2,0,0)", to="(pool_b4-east)", width=(8,8), height=8, depth=8, caption="Bottleneck"  ),
    to_connection( "pool_b4", "ccr_b5"),

    # ---------------------------------------------------------
    # Decoder (Upsampling)
    # ---------------------------------------------------------
    # Block-6: Up 1 (Input: 1024 -> Output: 256)
    *block_Unconv( name="b6", botton="ccr_b5", top='end_b6', s_filer=32,  n_filer=256, offset="(2.1,0,0)", size=(16,16,5.0), opacity=0.5 ),
    to_skip( of='ccr_b4', to='ccr_res_b6', pos=1.25),
    
    # Block-7: Up 2 (Input: 512 -> Output: 128)
    *block_Unconv( name="b7", botton="end_b6", top='end_b7', s_filer=64, n_filer=128, offset="(2.1,0,0)", size=(25,25,4.5), opacity=0.5 ),
    to_skip( of='ccr_b3', to='ccr_res_b7', pos=1.25),    
    
    # Block-8: Up 3 (Input: 256 -> Output: 64)
    *block_Unconv( name="b8", botton="end_b7", top='end_b8', s_filer=128, n_filer=64, offset="(2.1,0,0)", size=(32,32,3.5), opacity=0.5 ),
    to_skip( of='ccr_b2', to='ccr_res_b8', pos=1.25),    
    
    # Block-9: Up 4 (Input: 128 -> Output: 64)
    *block_Unconv( name="b9", botton="end_b8", top='end_b9', s_filer=256, n_filer=64,  offset="(2.1,0,0)", size=(40,40,2.5), opacity=0.5 ),
    to_skip( of='ccr_b1', to='ccr_res_b9', pos=1.25),
    
    # ---------------------------------------------------------
    # Output Head
    # ---------------------------------------------------------
    # Conv3d (64->3) + Tanh. Width=1 indicates low channel count.
    to_ConvSoftMax( name="out_layer", s_filer=256, offset="(0.75,0,0)", to="(end_b9-east)", width=1, height=40, depth=40, caption="Conv3D+Tanh" ),
    to_connection( "end_b9", "out_layer"),
     
    to_end() 
    ]

# Get current script filename (without extension)
try:
    namefile = str(sys.argv[0]).split('/')[-1].split('.')[0]
    # Set default name for interactive environments (Jupyter/Colab)
    if 'ipykernel' in namefile or namefile == '': 
        namefile = 'tex/unet3d_diagram'
except:
    namefile = 'tex/unet3d_diagram'
    
output_filename = namefile + '.tex'

# Generate Tex code
to_generate(arch, output_filename)

# === Print Output Path ===
current_dir = os.getcwd()
full_path = os.path.join(current_dir, output_filename)

print("="*60)
print(f"TeX file generated successfully!")
print(f"Filename: {output_filename}")
print(f"Full Path: {full_path}")
print("="*60)


\documentclass[border=8pt, multi, tikz]{standalone} 
\usepackage{import}
\subimport{../layers/}{init}
\usetikzlibrary{positioning}
\usetikzlibrary{3d} %for including external image 


\def\ConvColor{rgb:yellow,5;red,2.5;white,5}
\def\ConvReluColor{rgb:yellow,5;red,5;white,5}
\def\PoolColor{rgb:red,1;black,0.3}
\def\UnpoolColor{rgb:blue,2;green,1;black,0.3}
\def\FcColor{rgb:blue,5;red,2.5;white,5}
\def\FcReluColor{rgb:blue,5;red,5;white,4}
\def\SoftmaxColor{rgb:magenta,5;black,7}   
\def\SumColor{rgb:blue,5;green,15}


\newcommand{\copymidarrow}{\tikz \draw[-Stealth,line width=0.8mm,draw={rgb:blue,4;red,1;green,1;black,3}] (-0.3,0) -- ++(0.3,0);}

\begin{document}
\begin{tikzpicture}
\tikzstyle{connection}=[ultra thick,every node/.style={sloped,allow upside down},draw=\edgecolor,opacity=0.7]
\tikzstyle{copyconnection}=[ultra thick,every node/.style={sloped,allow upside down},draw={rgb:blue,4;red,1;green,1;black,3},opacity=0.7]


\node[canvas is zy plane at x=0] (input) at (-2,0,0) {\in

# Generate 2D U-Net Tex Code

In [14]:
# Define the architecture diagram list
arch = [ 
    to_head('..'), 
    to_cor(),
    to_begin(),
    
    # ---------------------------------------------------------
    # 1. Input Layer
    # ---------------------------------------------------------
    # Input: 15 Channels (e.g., 5 stacked RGB frames)
    # Width=3 makes it look slightly thicker to represent the stack
    to_input( '../inputpic/plant03_day01.png', to='(-2,0,0)', width=3, height=8, name="input" ),

    # ---------------------------------------------------------
    # 2. Encoder (Downsampling Path)
    # ---------------------------------------------------------
    # Level 1: Encoder (15 -> 64)
    to_ConvConvRelu( name='ccr_b1', s_filer=40, n_filer=(64,64), offset="(0,0,0)", to="(0,0,0)", width=(2,2), height=40, depth=40, caption="Encoder 1" ),
    to_Pool(name="pool_b1", offset="(0,0,0)", to="(ccr_b1-east)", width=1, height=32, depth=32, opacity=0.5),
    
    # Level 2: Down (64 -> 128)
    *block_2ConvPool( name='b2', botton='pool_b1', top='pool_b2', s_filer=32, n_filer=128, offset="(1,0,0)", size=(32,32,3.5), opacity=0.5 ),
    
    # Level 3: Down (128 -> 256)
    *block_2ConvPool( name='b3', botton='pool_b2', top='pool_b3', s_filer=25, n_filer=256, offset="(1,0,0)", size=(25,25,4.5), opacity=0.5 ),
    
    # Level 4: Down (256 -> 512)
    *block_2ConvPool( name='b4', botton='pool_b3', top='pool_b4', s_filer=16,  n_filer=512, offset="(1,0,0)", size=(16,16,5.5), opacity=0.5 ),

    # ---------------------------------------------------------
    # 3. Bottleneck
    # ---------------------------------------------------------
    # Level 5: Bottleneck (512 -> 512)
    to_ConvConvRelu( name='ccr_b5', s_filer=8, n_filer=(512,512), offset="(2,0,0)", to="(pool_b4-east)", width=(8,8), height=8, depth=8, caption="Bottleneck"  ),
    to_connection( "pool_b4", "ccr_b5"),

    # ---------------------------------------------------------
    # 4. Decoder (Upsampling Path)
    # ---------------------------------------------------------
    # Level 6: Up (1024 -> 256)
    # Concatenates with skip connection from Level 4
    *block_Unconv( name="b6", botton="ccr_b5", top='end_b6', s_filer=16,  n_filer=256, offset="(2.1,0,0)", size=(16,16,5.0), opacity=0.5 ),
    to_skip( of='ccr_b4', to='ccr_res_b6', pos=1.25),
    
    # Level 7: Up (512 -> 128)
    # Concatenates with skip connection from Level 3
    *block_Unconv( name="b7", botton="end_b6", top='end_b7', s_filer=25, n_filer=128, offset="(2.1,0,0)", size=(25,25,4.5), opacity=0.5 ),
    to_skip( of='ccr_b3', to='ccr_res_b7', pos=1.25),    
    
    # Level 8: Up (256 -> 64)
    # Concatenates with skip connection from Level 2
    *block_Unconv( name="b8", botton="end_b7", top='end_b8', s_filer=32, n_filer=64, offset="(2.1,0,0)", size=(32,32,3.5), opacity=0.5 ),
    to_skip( of='ccr_b2', to='ccr_res_b8', pos=1.25),    
    
    # Level 9: Up (128 -> 64)
    # Concatenates with skip connection from Level 1
    # Note: Output channels remain 64 here
    *block_Unconv( name="b9", botton="end_b8", top='end_b9', s_filer=40, n_filer=64,  offset="(2.1,0,0)", size=(40,40,2.5), opacity=0.5 ),
    to_skip( of='ccr_b1', to='ccr_res_b9', pos=1.25),
    
    # ---------------------------------------------------------
    # 5. Output Head
    # ---------------------------------------------------------
    # Output: Conv 1x1 + Tanh (64 -> 3 channels)
    to_ConvSoftMax( name="out_layer", s_filer=40, offset="(0.75,0,0)", to="(end_b9-east)", width=1, height=40, depth=40, caption="Conv 1x1 + Tanh" ),
    to_connection( "end_b9", "out_layer"),
     
    to_end() 
    ]

# === Automatic Filename Handling ===
try:
    namefile = str(sys.argv[0]).split('/')[-1].split('.')[0]
    # Set default name if running in Jupyter/Interactive mode
    if 'ipykernel' in namefile or namefile == '': 
        namefile = 'tex/unet2d_arch' 
except:
    namefile = 'tex/unet2d_arch'
    
output_filename = namefile + '.tex'

# Generate Tex Code
to_generate(arch, output_filename)

# === Print Confirmation ===
current_dir = os.getcwd()
full_path = os.path.join(current_dir, output_filename)

print("="*60)
print(f"2D U-Net Diagram Generated Successfully!")
print(f"Output File: {output_filename}")
print(f"Logic Flow: Input(15ch) -> Enc(64-512) -> Dec(256-64) -> Out(3ch)")
print("="*60)


\documentclass[border=8pt, multi, tikz]{standalone} 
\usepackage{import}
\subimport{../layers/}{init}
\usetikzlibrary{positioning}
\usetikzlibrary{3d} %for including external image 


\def\ConvColor{rgb:yellow,5;red,2.5;white,5}
\def\ConvReluColor{rgb:yellow,5;red,5;white,5}
\def\PoolColor{rgb:red,1;black,0.3}
\def\UnpoolColor{rgb:blue,2;green,1;black,0.3}
\def\FcColor{rgb:blue,5;red,2.5;white,5}
\def\FcReluColor{rgb:blue,5;red,5;white,4}
\def\SoftmaxColor{rgb:magenta,5;black,7}   
\def\SumColor{rgb:blue,5;green,15}


\newcommand{\copymidarrow}{\tikz \draw[-Stealth,line width=0.8mm,draw={rgb:blue,4;red,1;green,1;black,3}] (-0.3,0) -- ++(0.3,0);}

\begin{document}
\begin{tikzpicture}
\tikzstyle{connection}=[ultra thick,every node/.style={sloped,allow upside down},draw=\edgecolor,opacity=0.7]
\tikzstyle{copyconnection}=[ultra thick,every node/.style={sloped,allow upside down},draw={rgb:blue,4;red,1;green,1;black,3},opacity=0.7]


\node[canvas is zy plane at x=0] (input) at (-2,0,0) {\in

# ConvLSTM U-Net Model Architecture

In [ ]:
# Define the architecture
arch = [ 
    to_head('..'), 
    to_cor(),
    to_begin(),
    
    # ---------------------------------------------------------
    # 1. Input Sequence
    # ---------------------------------------------------------
    # Input: Sequence of images
    # Width=4 represents the time/frame stack
    to_input( '../inputpic/plant03_day01.png', to='(-2,0,0)', width=4, height=8, name="input" ),

    # ---------------------------------------------------------
    # 2. Encoder (Per-Frame Feature Extraction)
    # ---------------------------------------------------------
    
    # Level 1: Enc 1 (3 -> 64)
    # Note: n_filer uses a tuple (64,64) for correct rendering
    to_ConvConvRelu( name='ccr_e1', s_filer=40, n_filer=(64,64), offset="(0,0,0)", to="(0,0,0)", width=(2,2), height=40, depth=40, caption="Enc 1" ),
    
    # Level 2: Enc 2 (64 -> 128)
    # Downsampling (MaxPool + DoubleConv)
    to_Pool(name="pool_e1", offset="(0,0,0)", to="(ccr_e1-east)", width=1, height=32, depth=32, opacity=0.5),
    *block_2ConvPool( name='e2', botton='pool_e1', top='pool_e2', s_filer=32, n_filer=128, offset="(1,0,0)", size=(32,32,3.5), opacity=0.5 ),
    
    # Level 3: Enc 3 (128 -> 256)
    *block_2ConvPool( name='e3', botton='pool_e2', top='pool_e3', s_filer=24, n_filer=256, offset="(1,0,0)", size=(24,24,4.5), opacity=0.5 ),

    # ---------------------------------------------------------
    # 3. Temporal Bottleneck (ConvLSTM)
    # ---------------------------------------------------------
    # Bottleneck: ConvLSTM (256 -> 256)
    # Captures temporal dependencies between frames
    to_ConvConvRelu( name='lstm', s_filer=24, n_filer=(256,256), offset="(2,0,0)", to="(pool_e3-east)", width=(5,5), height=24, depth=24, caption="ConvLSTM (Time)" ),
    to_connection( "pool_e3", "lstm"),

    # ---------------------------------------------------------
    # 4. Decoder (Reconstruction)
    # ---------------------------------------------------------
    
    # Level 4: Up 1 (256+256 -> 128)
    # Skip connection from Enc 3
    *block_Unconv( name="d1", botton="lstm", top='end_d1', s_filer=32, n_filer=128, offset="(2.1,0,0)", size=(32,32,3.5), opacity=0.5 ),
    to_skip( of='ccr_e3', to='ccr_res_d1', pos=1.25),
    
    # Level 5: Up 2 (128+128 -> 64)
    # Skip connection from Enc 2
    *block_Unconv( name="d2", botton="end_d1", top='end_d2', s_filer=40, n_filer=64, offset="(2.1,0,0)", size=(40,40,2.5), opacity=0.5 ),
    to_skip( of='ccr_e2', to='ccr_res_d2', pos=1.25),
    
    # Level 6: Up 3 (64+64 -> 64)
    # Skip connection from Enc 1
    *block_Unconv( name="d3", botton="end_d2", top='end_d3', s_filer=40, n_filer=64, offset="(2.1,0,0)", size=(40,40,2.5), opacity=0.5 ),
    to_skip( of='ccr_e1', to='ccr_res_d3', pos=1.25),

    # ---------------------------------------------------------
    # 5. Output Head
    # ---------------------------------------------------------
    # Output: Prediction (Tanh)
    # Width=1 indicates single/low channel output
    to_ConvSoftMax( name="out_layer", s_filer=40, offset="(1.0,0,0)", to="(end_d3-east)", width=1, height=40, depth=40, caption="Pred (Tanh)" ),
    to_connection( "end_d3", "out_layer"),
     
    to_end() 
    ]

# === Generate File ===
try:
    namefile = str(sys.argv[0]).split('/')[-1].split('.')[0]
    # Set default filename for interactive/notebook environments
    if 'ipykernel' in namefile or namefile == '': 
        namefile = 'tex/convlstm_unet_fixed'
except:
    namefile = 'tex/convlstm_unet_fixed'
    
output_filename = namefile + '.tex'

to_generate(arch, output_filename)

# === Print Confirmation ===
print("="*60)
print(f"Corrected Architecture Code Generated: {output_filename}")
print("="*60)


\documentclass[border=8pt, multi, tikz]{standalone} 
\usepackage{import}
\subimport{../layers/}{init}
\usetikzlibrary{positioning}
\usetikzlibrary{3d} %for including external image 


\def\ConvColor{rgb:yellow,5;red,2.5;white,5}
\def\ConvReluColor{rgb:yellow,5;red,5;white,5}
\def\PoolColor{rgb:red,1;black,0.3}
\def\UnpoolColor{rgb:blue,2;green,1;black,0.3}
\def\FcColor{rgb:blue,5;red,2.5;white,5}
\def\FcReluColor{rgb:blue,5;red,5;white,4}
\def\SoftmaxColor{rgb:magenta,5;black,7}   
\def\SumColor{rgb:blue,5;green,15}


\newcommand{\copymidarrow}{\tikz \draw[-Stealth,line width=0.8mm,draw={rgb:blue,4;red,1;green,1;black,3}] (-0.3,0) -- ++(0.3,0);}

\begin{document}
\begin{tikzpicture}
\tikzstyle{connection}=[ultra thick,every node/.style={sloped,allow upside down},draw=\edgecolor,opacity=0.7]
\tikzstyle{copyconnection}=[ultra thick,every node/.style={sloped,allow upside down},draw={rgb:blue,4;red,1;green,1;black,3},opacity=0.7]


\node[canvas is zy plane at x=0] (input) at (-2,0,0) {\in